## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSBSBS'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename).filter("station like 'USC%'")
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1575
+-------------------+------------------+-------+---------+--------+--------+---------+-----------+------------------+--------------------+--------------------+--------+-----------+---------+------+--------------------+------+
|            coeff_1|           coeff_2|coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|               res_2|               res_3|res_mean|    station|total_var|undefs|              vector|  year|
+-------------------+------------------+-------+---------+--------+--------+---------+-----------+------------------+--------------------+--------------------+--------+-----------+---------+------+--------------------+------+
|0.06909685945706352|0.6838949647973293|    0.0|      2.1|BSBSBSBS| 25.3867| -80.5936|       SNWD|0.9898952161016833|2.650785772283967...|2.650785772283967...|Infinity|USC00087760|      0.0|     1|[00 00 00 00 00 0...|1950.0|
|0.06909685945706352|

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


,station,year,coeff_2
0,USC00087760,1950.0,0.683895
1,USC00087760,1951.0,0.683895
2,USC00087760,1952.0,0.683895
3,USC00087760,1953.0,0.683895
4,USC00087760,1954.0,0.683895


In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station,USC00080488,USC00080611,USC00080616,USC00080737,USC00080739,USC00080845,USC00080992,USC00081271,USC00081276,USC00081305,...,USC00088368,USC00088396,USC00088620,USC00088775,USC00088780,USC00088841,USC00089219,USC00089511,USC00089520,USC00089521
year,,,,,,,,,,,,,,,,,,,,,
1949.0,NaN,0.683895,0.683895,NaN,NaN,0.683895,NaN,0.683895,NaN,NaN,...,NaN,NaN,NaN,0.683895,0.683895,NaN,NaN,NaN,NaN,NaN
1950.0,NaN,0.683895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.683895,NaN,0.683895,0.683895,NaN,NaN,0.683895,NaN
1951.0,NaN,0.683895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.683895,...,NaN,NaN,0.683895,NaN,0.683895,0.683895,NaN,NaN,0.683895,NaN
1952.0,NaN,0.683895,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.683895,...,NaN,NaN,0.683895,NaN,0.683895,0.683895,NaN,NaN,0.683895,NaN
1953.0,NaN,0.683895,NaN,NaN,NaN,NaN,NaN,NaN,0.683895,0.683895,...,NaN,NaN,0.683895,NaN,0.683895,0.683895,NaN,NaN,0.683895,NaN
1954.0,NaN,0.683895,NaN,NaN,NaN,NaN,NaN,NaN,0.683895,0.683895,...,NaN,NaN,0.683895,NaN,0.683895,0.683895,NaN,NaN,0.683895,NaN
1955.0,NaN,0.683895,NaN,NaN,NaN,NaN,NaN,NaN,0.683895,0.683895,...,NaN,0.683895,0.683895,NaN,0.683895,0.683895,NaN,NaN,NaN,NaN
1956.0,NaN,0.683895,NaN,NaN,NaN,NaN,NaN,NaN,0.683895,0.683895,...,NaN,0.683895,0.683895,NaN,0.683895,0.683895,NaN,NaN,NaN,NaN
1957.0,NaN,0.683895,NaN,NaN,NaN,NaN,NaN,NaN,0.683895,0.683895,...,NaN,0.683895,0.683895,NaN,0.683895,0.683895,NaN,NaN,0.683895,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  0.683894964797
RMS removing mean-by-station=  2.55109828664e-16
RMS removing mean-by-year   =  5.10495675555e-17


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 0.683894964797
0 after removing mean by year    = 5.10495675555e-17
0 after removing mean by stations= 5.03801167101e-17
1 after removing mean by year    = 7.68785375606e-18
1 after removing mean by stations= 1.94166548033e-18
2 after removing mean by year    = 7.28239671171e-19
2 after removing mean by stations= 4.05647245222e-19
3 after removing mean by year    = 2.74087060008e-19
3 after removing mean by stations= 1.95421110559e-19
4 after removing mean by year    = 1.41142443799e-19
4 after removing mean by stations= 1.02270339197e-19
